In [ ]:
import os
import struct
import numpy as np
import pandas as pd

### reference
   - [stackoverflow](https://stackoverflow.com/questions/16444726/binary-representation-of-float-in-python-bits-not-hex)
   - [struct 라이브러리](https://docs.python.org/3/library/struct.html#format-strings)

In [ ]:
def binary(num,big_endian=True, bit_16=True):
    # big_endian : True 이면, big_endian format으로 출력, 아니면 little_endian format으로 출력
    # bit_16 : True 이면, 16bit float으로 출력, 아니면 32bit float으로 출력
    endian_format = ">" if big_endian else '<'
    bit_format = "e" if bit_16 else 'f'
    num_format = endian_format + bit_format
    struct.pack(num_format, num)
    return ''.join(bin(c).replace('0b', '').rjust(8, '0') for c in struct.pack(num_format, num)[::-1])
    #return ''.join(bin(c).replace('0b', '').rjust(8, '0') for c in struct.pack(num_format, num))

| float 16| sign(1) | expo(5) | remain(10) |
|---------|---------|---------|------------|
| float 32| sign(1) | expo(8) | remain(23) |

### tb_convert에 이용되는 테스트 데이터셋

In [ ]:
# 16bit - Big Endian value list
with open("../input/tb_convert_16be.txt",'w') as f:
    for i in range(-10,10):
        value = (1+2**(-3)+2**(-2)) * 2**(i)
        line = (binary(value,big_endian=True,bit_16=True))+'0'*16 + '\n'
        f.write(line)
        
# 16bit - Big Endian & 32bit - little Endian
df = pd.DataFrame(columns=['value','16bit-BigEndian','32bit-LittleEndian'])
for i in range(-10,10):
    value = (1+2**(-3)+2**(-2)) * 2**(i)
    be_16 = (binary(value,big_endian=True,bit_16=True))+'0'*16
    le_32 = binary(value,big_endian=False,bit_16=False)
    df = df.append({'value':value, "16bit-BigEndian":be_16,"32bit-LittleEndian":le_32},ignore_index=True)
df.to_csv("../answer/tb_convert_16be_answer.csv",index=False,float_format="%03.13f")

### tb_basic_pe에 이용되는 테스트 데이터셋

### 0. tb_basic_pe_ex0.txt
    => 수업시간에 준 input.txt

### 1. tb_basic_pe_ex1.txt

* value type : 32bit little-endian

* vector_size : 64

* 출력 형태 : 
2개의 동일 길이 벡터를 곱함

In [ ]:
def hex_to_float(h):
    return struct.unpack('>f',bytes.fromhex(h))[0]

def float_to_hex(f):
    return hex(struct.unpack('>I', struct.pack('>f', f))[0]).replace('0x','')

def hex_to_bin(h):
    return bin(int(h, 16))[2:].zfill(32)

def bin_to_hex(b):
    return hex(int(b,2))

In [ ]:
first_vector = np.linspace(0.5,2.5,num=64)
second_vector = np.linspace(1,3,num=64)

In [ ]:
result = np.sum(first_vector*second_vector)
print("return value:",result,"\nhex value : ",float_to_hex(result))

In [ ]:
with open("../input/tb_basic_pe_ex1.txt",'w') as file:
    for f in first_vector:
        file.write(float_to_hex(f)+'\n')
    for f in second_vector:
        file.write(float_to_hex(f)+'\n')

### tb_convert_pe에 이용되는 테스트 데이터셋

reference: https://stackoverflow.com/questions/8751653/how-to-convert-a-binary-string-into-a-float-value

### 1. tb_convert_pe_ex1.txt

In [ ]:
# default : little-endian
def bin_to_float(binary):
    return struct.unpack('<f',struct.pack('<I', int(binary, 2)))[0]
# default : little-endian
def float_to_bin(num):
    return bin(struct.unpack('!I', struct.pack('!f', num))[0])[2:].zfill(32)

In [ ]:
first_vector = np.linspace(0.5,2.5,num=64)
second_vector = np.linspace(1,3,num=64)

In [ ]:
with open("../input/tb_convert_pe_ex1.txt",'w') as file:
    for value in first_vector:        
        le_32 = binary(value,big_endian=True,bit_16=True)
        file.write(le_32+'0'*16+"\n")
    for value in second_vector:
        le_32 = binary(value,big_endian=True,bit_16=True)
        file.write(le_32+'0'*16+"\n")

In [ ]:
result = np.sum(first_vector*second_vector)

print("return value:",result,"\nhex value : ",
      binary(result,big_endian=False,bit_16=False))

print("zynq - bit: ",'01000011010101011111111011101011')
print("zynq - result : ",bin_to_float('01000011010101011111111011101011'))

### tb_matrix_pe에 이용되는 테스트 데이터셋

#### mini-size 버전

In [ ]:
# input_value
matrix = np.zeros((16,16),dtype=float)
for i in range(1,16+1):
    matrix[i-1] = np.linspace(0.5*i,2.5*i,num=16)

vector = np.linspace(1,3,num=16)

# result 
result = np.sum(matrix*vector,axis=1)
for i in range(16):
    print("[{:02}]번째 결과 : {}[{}]".
          format(i,binary(result[i],big_endian=False,bit_16=False),result[i]))

In [ ]:
# input file
with open("../input/tb_matrix_pe_ex0.txt",'w') as file:
    for i in range(16):
        for j in range(16):
            be_16 = binary(matrix[i,j],big_endian=True,bit_16=True)
            print("matrix[{},{}] : {}".format(i,j,be_16))
            file.write(be_16+'0'*16+"\n")

    for idx, value in enumerate(vector):
        be_16 = binary(value,big_endian=True,bit_16=True)
        print("vector[{}]  : {}".format(idx, be_16))
        file.write(be_16+'0'*16+"\n")
        
# result
df = pd.DataFrame(columns=['index','value','bit'])

for i in range(16):
    df = df.append({
        "index":"{:02}".format(i),
        "value":result[i],
        "bit":binary(result[i],big_endian=False,bit_16=False)
    },ignore_index=True)
    
df.to_csv("../answer/tb_matrix_pe_ex0_answer.csv",index=False,float_format="%03.13f")

In [ ]:
result_df = pd.read_csv("../answer/tb_matrix_pe_ex0_out.txt",header=None,names=['result'])
df['result_value'] = result_df.result[:16].apply(lambda x : bin_to_float(x))
df['result_bit'] = result_df.result[:16]
df['error_rate'] = np.abs(df.value - df.result_value) / df.value * 100

In [ ]:
df

#### 64x64 size 버젼

In [ ]:
# input_value
matrix = np.zeros((64,64),dtype=float)
for i in range(1,64+1):
    matrix[i-1] = np.linspace(0.5*i,2.5*i,num=64)

vector = np.linspace(1,3,num=64)

# result 
result = np.sum(matrix*vector,axis=1)
for i in range(64):
    print("[{:02}]번째 결과 : {}[{}]".
          format(i,binary(result[i],big_endian=False,bit_16=False),result[i]))

In [ ]:
with open("../input/tb_matrix_pe_ex1.txt",'w') as file:
    for i in range(64):
        for j in range(64):
            be_16 = binary(matrix[i,j],big_endian=True,bit_16=True)
            print("matrix[{},{}] : {}".format(i,j,be_16))
            file.write(be_16+'0'*16+"\n")

    for idx, value in enumerate(vector):
        be_16 = binary(value,big_endian=True,bit_16=True)
        print("vector[{}]  : {}".format(idx, be_16))
        file.write(be_16+'0'*16+"\n")
        
# result
df = pd.DataFrame(columns=['index','value','bit'])

for i in range(64):
    df = df.append({
        "index":"{:02}".format(i),
        "value":result[i],
        "bit":binary(result[i],big_endian=False,bit_16=False)
    },ignore_index=True)
    
df.to_csv("../answer/tb_matrix_pe_ex1_answer.csv",index=False,float_format="%03.13f")

### 오차율 계산

In [ ]:
result_df = pd.read_csv("./tb_matrix_pe_ex1_out.txt",header=None,names=['result'])
df['result_value'] = result_df.result[:64].apply(lambda x : bin_to_float(x))
df['result_bit'] = result_df.result[:64]
df['error_rate'] = np.abs(df.value - df.result_value) / df.value * 100

------------------------------------------------------
------------------------------------------------------

In [ ]:
input_df = pd.read_csv("../input/input.txt",header=None,names=['input'])
input_df = input_df.input.apply(lambda num : binary(num,big_endian=True, bit_16=True)+'0'*16)
input_df.to_csv("../input/tb_matrix_pe_ex2.txt",index=False,header=False)

In [ ]:
df = pd.read_csv("../input/input.txt",header=None,names=['input'])
mat = df[:64*64].values.reshape((64,64))
vec = df[64*64:].values

mat = mat.astype(np.float16).astype(np.float32)
vec = vec.astype(np.float16).astype(np.float32)

In [ ]:
result =np.sum(mat * vec.reshape((1,64)),axis=1,dtype=np.float)

In [ ]:
result_df = pd.read_csv("../answer/tb_matrix_pe_ex2_out.txt",header=None,names=['result'])

df = pd.DataFrame()
df['value'] = result
df['result_value'] = result_df.result[:64].apply(lambda x : bin_to_float(x))
df['result_bit'] = result_df.result[:64]
df['error_rate'] = np.abs(df.value - df.result_value) / df.value * 100

In [ ]:
df